In [6]:
import os
import itertools
import torch
import lightning.pytorch as pl
from lightning.pytorch.callbacks.progress import TQDMProgressBar
import torchmetrics
import torch.nn.functional as F
import pytorchvideo.data

import matplotlib.pyplot as plt
import wandb

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    RemoveKey,
    ShortSideScale,
    UniformTemporalSubsample,
)

from torchvision.transforms import (
    CenterCrop,
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize,
)


from transformers import AutoImageProcessor, TimesformerForVideoClassification

#### Limit Dataset

 To ensure a constant number of samples are retrieved from the dataset

In [2]:
class LimitDataset(torch.utils.data.Dataset):

    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset
        self.dataset_iter = itertools.chain.from_iterable(
            itertools.repeat(iter(dataset), 2)
        )

    def __getitem__(self, index):
        return next(self.dataset_iter)

    def __len__(self):
        return self.dataset.num_videos

#### Fly Data Module

In [3]:
class FlyDataModule(pl.LightningDataModule):

    def __init__(self, args):
        self.args = args
        super().__init__()

    def _make_transforms(self, mode: str):
        return Compose([self._video_transform(mode)])

    def _video_transform(self, mode: str):
        return ApplyTransformToKey(
            key="video",
            transform=Compose(
                [
                    UniformTemporalSubsample(self.args["num_frames_to_sample"]),
                    Lambda(lambda x: x / 255.0),
                    Normalize(self.args["video_means"], self.args["video_stds"]),
                ]
                + (
                    [
                        RandomShortSideScale(
                            min_size=self.args["video_min_short_side_scale"],
                            max_size=self.args["video_max_short_side_scale"],
                        ),
                        RandomCrop(self.args["crop_size"]),
                    ]
                    if mode == "train"
                    else [
                        ShortSideScale(self.args["video_min_short_side_scale"]),
                        CenterCrop(self.args["crop_size"]),
                    ]
                )
            ),
        )

    def train_dataloader(self):

        train_transform = self._make_transforms(mode="train")

        self.train_dataset = LimitDataset(
            pytorchvideo.data.labeled_video_dataset(
                data_path=self.args["train_data_path"],
                clip_sampler=pytorchvideo.data.make_clip_sampler('uniform', self.args["clip_duration"]), # Experiment olarak random da denenebilir
                transform=train_transform,
                video_path_prefix=self.args["video_path_prefix"], # could be '' I think
                decode_audio=False
            )
        )

        return torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=self.args["batch_size"]
        )

    def val_dataloader(self):
        
        val_transform = self._make_transforms(mode="val")

        self.val_dataset = LimitDataset(
            pytorchvideo.data.labeled_video_dataset(
                data_path=self.args["val_data_path"],
                clip_sampler=pytorchvideo.data.make_clip_sampler('uniform', self.args["clip_duration"]), # Experiment olarak random da denenebilir
                transform=val_transform,
                video_path_prefix=self.args["video_path_prefix"], # could be '' I think
                decode_audio=False
            )
        )

        return torch.utils.data.DataLoader(
            self.val_dataset,
            batch_size=self.args["batch_size"]
        )

#### Lightning Module

In [16]:
class VideoClassificationLightningModule(pl.LightningModule):
    def __init__(self, model, args):
        super().__init__()

        self.args = args
        self.model = model
        self.dataloader_length = 0
        self.classes = ['Feeding', 'Grooming', 'Pumping']

        self.save_hyperparameters("args")
        
        # For logging outputs
        self.epoch_logits = []
        self.epoch_incorrect_samples = []

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        return self._common_step(batch, batch_idx, stage='train')

    def validation_step(self, batch, batch_idx):
        self._common_step(batch, batch_idx, stage='val')
    
    def _common_step(self, batch, batch_idx, stage='train'):
        X, y = batch['video'], batch['label']

        output = self(X.permute(0, 2, 1, 3, 4)) # (8, 3, 16, 224, 224) -> (8, 16, 3, 224, 224)

        loss = F.cross_entropy(output.logits, y)
        acc = torchmetrics.functional.accuracy(output.logits, y, task="multiclass", num_classes=3)

        self.log(
            f"{stage}_loss", loss.item(), batch_size=self.args["batch_size"], on_step=False, on_epoch=True, prog_bar=True
        )
        self.log(
            f"{stage}_acc", acc, batch_size=self.args["batch_size"], on_step=False, on_epoch=True, prog_bar=True
        )
        if stage == 'train':
            return loss
        elif stage == 'val':
            _, predictions = torch.max(output.logits, dim=1)
            incorrect_samples = X[predictions != y]  # Get incorrect samples
            self.epoch_logits.extend(output.logits)
            if len(self.epoch_incorrect_samples) < 1:
                self.epoch_incorrect_samples.extend(incorrect_samples[0])

    def on_validation_epoch_end(self):
        #dummy_input = torch.zeros((1, 8, 3, 224, 224), device=self.device)
        #model_filename = "model_ckpt.onnx"
        #torch.onnx.export(self, dummy_input, model_filename, opset_version=11)
        #artifact = wandb.Artifact(name="model.ckpt", type="model")
        #artifact.add_file(model_filename)
        #self.logger.experiment.log_artifact(artifact)

        flattened_logits = torch.flatten(torch.cat(self.epoch_logits))
        incorrect_samples = self.epoch_incorrect_samples[0]

        print('false_predictions:', incorrect_samples.shape)
        print('logits:', flattened_logits)
        print('global_step:', self.global_step)
        
        self.epoch_logits.clear()
        self.epoch_incorrect_samples.clear()

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(),
            lr=self.args["lr"],
        )
        return [optimizer]

#### Config

In [18]:
def create_preprocessor_config(model, image_processor, sample_rate=8, fps=30):

    mean = image_processor.image_mean
    std = image_processor.image_std

    if "shortest_edge" in image_processor.size:
        height = width = image_processor.size["shortest_edge"]
    else:
        height = image_processor.size["height"]
        width = image_processor.size["width"]

    crop_size = (height, width)

    num_frames_to_sample = model.config.num_frames # 16 for VideoMAE
    clip_duration = num_frames_to_sample * sample_rate / fps
    print('Clip Duration:', clip_duration, 'seconds')

    return {
        "image_mean" : mean,
        "image_std" : std,
        "crop_size" : crop_size,
        "num_frames_to_sample" : num_frames_to_sample,
        "clip_duration": clip_duration
    }

In [8]:
def get_timesformer_model(ckpt, label2id, id2label, num_frames):
    return TimesformerForVideoClassification.from_pretrained(
        ckpt,
        label2id=label2id,
        id2label=id2label,
        ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
        num_frames = num_frames
    )

In [11]:
# PATH INFO
PROJ_DIR = '/Users/mpekey/Desktop/FlyVideo'
TRAIN_DATA_PATH = os.path.join(PROJ_DIR, 'FlyTrainingData', 'Train')
VAL_DATA_PATH = os.path.join(PROJ_DIR, 'FlyTrainingData', 'Validation')


# DATASET INFO
class_labels = ['Feeding', 'Grooming', 'Pumping']
label2id = {label: i for i, label in enumerate(class_labels)}
id2label = {i: label for label, i in label2id.items()}

image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
model = get_timesformer_model(ckpt="facebook/timesformer-base-finetuned-k400",
                                  label2id=label2id,
                                  id2label=id2label,
                                  num_frames=8)

# Freeze the model    
for param in model.timesformer.parameters():
    param.requires_grad = False


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.



















































Some weights of TimesformerForVideoClassification were not initialized from the model checkpoint at facebook/timesformer-base-finetuned-k400 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([400, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([400]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Clip Duration: 4.266666666666667 seconds


In [23]:
# Create Arguments
model_args = create_preprocessor_config(model, 
                                        image_processor, 
                                        sample_rate=16, 
                                        fps=30)

args = {
    # Data
    "train_data_path" : TRAIN_DATA_PATH,
    "val_data_path" : VAL_DATA_PATH,
    "lr" : 0.001,
    "max_epochs" : 1,
    "batch_size" : 16,
    "video_path_prefix" : '',
    "video_min_short_side_scale" : 256,
    "video_max_short_side_scale" : 320,
    "clip_duration" : model_args["clip_duration"],
    "crop_size" : model_args["crop_size"],
    "num_frames_to_sample": model_args["num_frames_to_sample"],
    "video_means" : model_args["image_mean"],
    "video_stds" : model_args["image_std"],
    "sample_rate": 16,
    "fps":30,
    "num_frames":8
}

Clip Duration: 4.266666666666667 seconds


#### Training

In [24]:
trainer = pl.Trainer(
        max_epochs=args["max_epochs"],
        callbacks=[TQDMProgressBar(refresh_rate=args["batch_size"])],
        accelerator="gpu" if torch.cuda.is_available() else "auto",
        log_every_n_steps=40
    )
classification_module = VideoClassificationLightningModule(model, args)
data_module = FlyDataModule(args)
trainer.fit(classification_module, data_module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /Users/mpekey/Desktop/FlyVideo/notebooks/lightning_logs

  | Name  | Type                              | Params
------------------------------------------------------------
0 | model | TimesformerForVideoClassification | 121 M 
------------------------------------------------------------
2.3 K     Trainable params
121 M     Non-trainable params
121 M     Total params
485.044   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/mpekey/miniconda3/envs/lightning/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/mpekey/miniconda3/envs/lightning/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
